In [ ]:
from fbprophet import Prophet
import pandas as pd
import numpy as np
from fbprophet.plot import plot_plotly, plot_components_plotly
import plotly.graph_objects as go

In [ ]:
def get_covid_data():
    
    #get the latest data from OxCGRT
    DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
    full_df = pd.read_csv(DATA_URL,
                    parse_dates=['Date'],
                    encoding="ISO-8859-1",
                    dtype={"RegionName": str,
                           "RegionCode":str,
                           "CountryName":str,
                           "CountryCode":str},
                    error_bad_lines=False)

    #add new cases and new deaths columns

    return full_df

def mean_percent_error(y_test, y_hat):
    error = np.abs(y_test - y_hat)
    percent_error = error/y_test
    mean_percent_error = percent_error.sum() / len(y_test)
    return mean_percent_error

def find_best_regressors(df, train_df, test_df):
    todrop = [c for c in df.columns if c in ['ds','y','NewCases','NewDeaths', 'ConfirmedCases','ConfirmedDeaths']]
    regressors = df.columns.drop(todrop)

    keepers = []
    trials = pd.DataFrame(columns = ['regressors','MAPE'])
    improving = True
    while improving:
        best = None
        improving = False

        print(f'current keepers are {keepers}')
        for regressor in regressors:
            keepers.append(regressor)
            m = pr.Prophet(seasonality_mode = 'multiplicative',
                            yearly_seasonality = False, 
                            daily_seasonality = False, 
                            weekly_seasonality = True)
            m.add_country_holidays(country_name='US')
            for keeper in keepers:
                m.add_regressor(keeper)
            m.fit(train_df)
            future = m.make_future_dataframe(periods=len(test_df))
            future = pd.merge(future,df[['ds'] + keepers].reset_index(drop=True),how = 'outer', on = 'ds')
            forecast = m.predict(future)
            prophet_mape = mean_percent_error(test_df['y'].values, forecast['yhat'][-len(test_df):].values)
            trials = trials.append({'regressors':f'{keepers}','MAPE':prophet_mape}, ignore_index=True)
            #MAPE has improved
            if prophet_mape == trials['MAPE'].min():
                improving = True
                best = regressor
            keepers.pop()
        if best:
            keepers.append(best)
        if improving:
            regressors = regressors.drop(best)
    return keepers

In [ ]:
division = 'state'
region = 'Washington'
prediction = 'ConfirmedCases'

full_df = get_covid_data()
if division == 'country':
    df = full_df[(full_df['Jurisdiction'] == 'NAT_TOTAL') & (full_df['CountryName'] == region)][:-1]
elif division == 'state':
    df = full_df[(full_df['Jurisdiction'] == 'STATE_TOTAL') & (full_df['RegionName'] == region)][:-1]

df = df.iloc[:,5:]
cols = [c for c in df.columns if (c.lower()[-10:] != 'fordisplay') and (c.lower()[-4:] != 'flag')]
df = df[cols]
df = df.dropna(how='all', axis=1)
df = df.fillna(method = 'ffill')
df = df.fillna(0)
df = df.rename(columns = {'Date':'ds',
                          prediction:'y'
                          })
df = df.drop(columns = [col for col in df.columns if col in ['ConfirmedDeaths','ConfirmedCases']])
train_df = df[df['ds'] < '2020-12-01']
test_df = df[df['ds'] >= '2020-12-01']

keepers = df[test_df.columns.drop(['y'])]
# keepers['C1_School closing'] = 
# keepers['C2_Workplace closing'] = 
# keepers['C3_Cancel public events'] = 
# keepers['C4_Restrictions on gatherings'] = 
# keepers['C5_Close public transport'] = 
# keepers['C6_Stay at home requirements'] = 
# keepers['C7_Restrictions on internal movement'] = 
# keepers['C8_International travel controls'] = 
# keepers['E1_Income support'] = 
# keepers['E2_Debt/contract relief'] = 
# keepers['E3_Fiscal measures'] = 
# keepers['E4_International support'] = 
# keepers['H1_Public information campaigns'] = 
# keepers['H2_Testing policy'] = 
# keepers['H3_Contact tracing'] = 
# keepers['H4_Emergency investment in healthcare'] = 
# keepers['H5_Investment in vaccines'] = 
# keepers['H6_Facial Coverings'] = 
# keepers.loc[keepers['ds'] >= '2020-12-01','H7_Vaccination policy'] = 0
# keepers['StringencyIndex'] = 
# keepers['StringencyLegacyIndex'] = 
# keepers.loc[keepers['ds'] >= '2020-12-01','GovernmentResponseIndex'] = 62.22
# keepers.loc[keepers['ds'] >= '2020-12-01','ContainmentHealthIndex'] = 60.26
# keepers['EconomicSupportIndex'] = 
# keepers = keepers[['GovernmentResponseIndex','ContainmentHealthIndex','ds']]


## Actual Intervention Strategy

In [ ]:
# keepers = find_best_regressors(df, train_df, test_df)
# keepers = [col for col in df.columns if "_" in col]
# keepers.loc[keepers['ds'] >= '2020-12-01','GovernmentResponseIndex'] = 62.22
# keepers.loc[keepers['ds'] >= '2020-12-01','ContainmentHealthIndex'] = 60.26

#Actual government response indices
m = Prophet(seasonality_mode = 'multiplicative',
                yearly_seasonality = False, 
                daily_seasonality = False, 
                weekly_seasonality = True)
m.add_country_holidays(country_name='US')
for keeper in keepers.columns.drop('ds'):
    m.add_regressor(keeper)
m.fit(train_df)
future = m.make_future_dataframe(periods=len(test_df))

future = pd.merge(future, keepers, how = 'outer', on = 'ds')

forecast = m.predict(future)
prophet_mape = mean_percent_error(test_df['y'].values, forecast['yhat'][-len(test_df):].values)
print(f'MAPE is {prophet_mape}')

#Freeze Government Response Indices at 2020-12-01
keepers_frozen = keepers.copy()
keepers_frozen.loc[keepers_frozen['ds'] >= '2020-12-01','GovernmentResponseIndex'] \
                = df.loc[df['ds'] == '2020-12-01','GovernmentResponseIndex'].values[0]
keepers_frozen.loc[keepers_frozen['ds'] >= '2020-12-01','ContainmentHealthIndex'] \
                = df.loc[df['ds'] == '2020-12-01','ContainmentHealthIndex'].values[0]
m2 = Prophet(seasonality_mode = 'multiplicative',
                yearly_seasonality = False, 
                daily_seasonality = False, 
                weekly_seasonality = True)
m2.add_country_holidays(country_name='US')
for keeper in keepers_frozen.columns.drop('ds'):
    m2.add_regressor(keeper)
m2.fit(train_df)
future_frozen = m.make_future_dataframe(periods=len(test_df))

future_frozen = pd.merge(future_frozen, keepers_frozen, how = 'outer', on = 'ds')

forecast_frozen = m.predict(future_frozen)
prophet_mape_frozen = mean_percent_error(test_df['y'].values, forecast_frozen['yhat'][-len(test_df):].values)
print(f'frozen MAPE is {prophet_mape_frozen}')

#Increase Government Response Indices by 10
keepers_increased = keepers.copy()
keepers_increased.loc[keepers_increased['ds'] >= '2020-12-01','GovernmentResponseIndex'] \
                = df.loc[df['ds'] == '2020-12-01','GovernmentResponseIndex'].values[0] +10
keepers_increased.loc[keepers_increased['ds'] >= '2020-12-01','ContainmentHealthIndex'] \
                = df.loc[df['ds'] == '2020-12-01','ContainmentHealthIndex'].values[0] +10
m3 = Prophet(seasonality_mode = 'multiplicative',
                yearly_seasonality = False, 
                daily_seasonality = False, 
                weekly_seasonality = True)
m3.add_country_holidays(country_name='US')
for keeper in keepers_increased.columns.drop('ds'):
    m3.add_regressor(keeper)
m3.fit(train_df)
future_increased = m.make_future_dataframe(periods=len(test_df))

future_increased = pd.merge(future_increased, keepers_increased, how = 'outer', on = 'ds')

forecast_increased = m.predict(future_increased)
prophet_mape = mean_percent_error(test_df['y'].values, forecast_increased['yhat'][-len(test_df):].values)
print(f'increase MAPE is {prophet_mape}')


fig = plot_plotly(m, forecast, changepoints = False, xlabel="Date", 
                  uncertainty = True,
                  ylabel=prediction, plot_cap=True)
fig.add_trace(go.Scatter(x=test_df['ds'], y=test_df['y'], 
                         mode = 'markers',
                         marker=go.scatter.Marker(color='green', size = 4),
                         name = f'True'
                         ))
fig.add_trace(go.Scatter(x=forecast_frozen['ds'], y =forecast_frozen['yhat'],
                         marker=go.scatter.Marker(color='yellow', size = 4),
                         name = f'prediction if frozen'
                         ))
fig.add_trace(go.Scatter(x=forecast_increased['ds'], y=forecast_increased['yhat'], 
                         mode = 'lines',
                         marker=go.scatter.Marker(color='red', size = 4),
                         name = f'prediction if increased'
                         ))
fig.layout.title = {'text': f'True and Predicted {prediction}'}
fig.update_layout(showlegend=True)
fig.show()

## Model is reversing causitive relationship between intervention and cases.  If interventions increase, model predicts cases increase.  In fact, it's more likely that governments increase interventions in response to rising cases.